# Using AngularStates and different spin coupling schemes

In [1]:
from ryd_numerov.angular import AngularStateFJ, AngularStateJJ, AngularStateLS

For angular states we always store the nuclear spin `i_c` (0 if no hyperfine states should be considered), the core electron spin `s_c` (0 for Alkali atoms, 1/2 for Alkaline earth atoms) and orbital momentum `l_c` (0 by default) and the rydberg electron spin `s_r = 0.5`  and orbital momentum `l_r`.
These spins can couple via different coupling schemes to a total spin momentum `f_tot` with magnetic quantum number `m`.
The different coupling schemes define in addition the following quantum numbers:
- `AngularStateLS`: the total orbital momentum `(l_c, l_r)l_tot`, the total spin `(s_c, s_r)s_tot`, and the total angular momentum `(l_tot, s_tot)j_tot`
- `AngularStateJJ`: the angular momentum of the core electron `(l_c, s_c)j_c`, the angular momentum of the rydberg electron `(s_r, l_r)j_r`, and the total angular momentum `(j_c, j_r)j_tot`
- `AngularStateFJ`: the angular momentum of the core electron `(l_c, s_c)j_c`, the angular momentum of the rydberg electron `(s_r, l_r)j_r`, and the total spin momentum of the core electron `(j_c, i_c)f_c`

To create an angular state, you can simply call the respective class and specify the needed quantum numbers.
Note, that you only have to specify as many quantum numbers as needed to uniquely define the state, all other quantum numbers will then be determined automatically.
Furthermore, you can specify the atomic species to automatically set the nuclear spin `i_c` and the core electron spin `s_c` (`s_c` will always be set to 0 by default).

The magnetic quantum number `m` is optional, and only needed if you want to calculate concrete matrix elements.
If you don't specify `m`, you still can calculate reduced matrix elements as well as reduced overlaps between two angular states.

In [2]:
state1 = AngularStateLS(s_tot=0, l_r=0, j_tot=0, species="Yb173")
print(f"{state1=}")
state2 = AngularStateJJ(j_tot=0, l_r=0, f_tot=2.5, species="Yb173")
print(f"{state2=}")
state3 = AngularStateFJ(f_c=2, l_r=0, f_tot=2.5, species="Yb173")
print(f"{state3=}")

state1=AngularStateLS(i_c=2.5, s_c=0.5, l_c=0, s_r=0.5, l_r=0, s_tot=0, l_tot=0, j_tot=0, f_tot=2.5, species='Yb173')
state2=AngularStateJJ(i_c=2.5, s_c=0.5, l_c=0, s_r=0.5, l_r=0, j_c=0.5, j_r=0.5, j_tot=0, f_tot=2.5, species='Yb173')
state3=AngularStateFJ(i_c=2.5, s_c=0.5, l_c=0, s_r=0.5, l_r=0, j_c=0.5, f_c=2, j_r=0.5, f_tot=2.5, species='Yb173')


Calculating overlaps between two angular states of different coupling schemes is as simply as calling the `calc_reduced_overlap` method.
Note, that this method ignores any given magnetic quantum numbers `m` (if specified).
This method will automatically check the coupling schemes of the two states and calculate the needed Wigner6j and Wigner9j symbols.

In [3]:
print(f"{state1.calc_reduced_overlap(state2)=}")
print(f"{state1.calc_reduced_overlap(state3)=}")
print(f"{state2.calc_reduced_overlap(state3)=}")

state1.calc_reduced_overlap(state2)=np.float64(1.0)
state1.calc_reduced_overlap(state3)=np.float64(-0.6454972243679028)
state2.calc_reduced_overlap(state3)=np.float64(-0.6454972243679028)


You can also convert between the different coupling schemes by using the `to_ls`, `to_jj`, and `to_fj` methods.
This will return a `SuperpositionState` object, which contains a list of `AngularState` objects and the respective coefficients.

In [4]:
print(state1.to_jj())
print(state1.to_fj())

1.0*JJ(i_c=2.5, s_c=0.5, l_c=0, s_r=0.5, l_r=0, j_c=0.5, j_r=0.5, j_tot=0, f_tot=2.5, species='Yb173')
-0.6454972243679028*FJ(i_c=2.5, s_c=0.5, l_c=0, s_r=0.5, l_r=0, j_c=0.5, f_c=2.0, j_r=0.5, f_tot=2.5, species='Yb173'), 0.7637626158259734*FJ(i_c=2.5, s_c=0.5, l_c=0, s_r=0.5, l_r=0, j_c=0.5, f_c=3.0, j_r=0.5, f_tot=2.5, species='Yb173')


The `SuperpositionState` class also provides methods to calculate expectation and standard deviations of quantum numbers: 

In [5]:
state = AngularStateFJ(f_c=2, l_r=1, j_r=1.5, f_tot=2.5, species="Yb173")

state_as_fj = state.to_fj()
exp_f_c = state_as_fj.exp_q("f_c")
std_f_c = state_as_fj.std_q("f_c")

state_as_ls = state.to_ls()
exp_s_tot = state_as_ls.exp_q("s_tot")
std_s_tot = state_as_ls.std_q("s_tot")

print(state)
print(f"{exp_f_c=}, {std_f_c=}")
print(f"{exp_s_tot=}, {std_s_tot=}")

FJ(i_c=2.5, s_c=0.5, l_c=0, s_r=0.5, l_r=1, j_c=0.5, f_c=2, j_r=1.5, f_tot=2.5, species='Yb173')
exp_f_c=np.int64(2), std_f_c=0
exp_s_tot=np.float64(0.7777777777777777), std_s_tot=np.float64(0.41573970964154916)
